## Netflix Eureka
Is a service discovery solution helpful in client-side load balancing scenarios, though it can be used for other purpose as well, for example in *Spring Cloud Gateway*.  

### Components
The Eureka ecosystem consists of the following actors:
- **Eureka Server:** a server where microservices can register themselves so others can discover them
- **Eureka Service:** any application that can be found in the Eureka Server's registry and is discoverable by others
- **Eureka Instance:** a single instance of a service (a Eureka Service can have multiple instances?)
- **Eureka Client:** any application that can discover services OR registers itself to the Eureka server

### Architecture
The digram below explains the Eureka Client/ Server setups fairly well:  

<img src="https://github.com/Netflix/eureka/raw/master/images/eureka_architecture.png"/>

To ensure high availability, the Eureka Server is replicated across zones (and also regions). More on this later.

### Configuration
Since Eureka is a client/server model, it accepts different sets of configuration for client and for the server. By default, Eureka searches for property file `eureka-client.properties` or `eureka-server.properties` in the classpath. Since a Eureka Server would also act as a client (to communicate with other Eureka Servers), it would need both configurations.

## Spring Cloud Netflix
Provides an abstraction over Netflix Eureka. To include the client component, add the following to the dependency list:
```
implementation 'org.springframework.cloud:spring-cloud-starter-netflix-eureka-client'
```

To include the server component,
```
implementation 'org.springframework.cloud:spring-cloud-starter-netflix-eureka-server'
```

## Eureka Client
To register a service, with the Eureka Server, include the client dependecy as shown above and add in the following property in `application.yml`:  
```yml
eureka:
  client:
    serviceUrl:
      defaultZone: http://localhost:8761/eureka/
```

The URL listed as the value of *defaultZone* property is the URL of the Eureka Server. *defaultZone* is a magic string fallback value that provides the service URL for any client that does not express a preference (in other words, it is a useful default).

We can also specify a comma separated list of URLs as the value, the client will use the second URL only if it is unable to connect to the first one.

### MultiZone Setup
--- MAY NOT BE ACCURATE ---  
consider the following scenario:
- there are two zones each hosting two services - product and billing
- each zone (us-east-1a, us-east-1b) also have a Eureka Server running

<img src="./images/zone_repl.png" />

How do we configure the client and server in this case while requiring, each client talks to the server in its zone only? Both billing and product service will have configuration like:
```yml
eureka:
  client:
    prefer-same-zone-eureka: true
    service-url:
      us-east-1a: http://localhost:8761/eureka/
      us-east-1b: http://127.0.0.1:8762/eureka/
  instance:
    metadataMap.zone: ${ZONE}
```

Where `${ZONE}` would resolve to either us-east-1a or us-east-1b depending upon in which zone the product or billing service got deployed to. The server would also need to be zone aware:
```yml
eureka:
  client:
    prefer-same-zone-eureka: true
    service-url:
      us-east-1a: http://localhost:8761/eureka/
      us-east-1b: http://127.0.0.1:8762/eureka/
  instance:
    metadataMap.zone: ${ZONE}
```

### Registering Service
If `eureka.client.register-with-eureka` is set to true, an instance registers with a Eureka server using the given URL. Multiple instances can belong to the same service. An instance is identified in Eureka by `eureka.instance.instanceId` or, if not present, `eureka.instance.metadataMap.instanceId`. If both are not specified, it defaults to `CLIENT IP:PORT`. The instances find each other using `eureka.instance.appName` (identifier for a service), which Spring Cloud defaults to `spring.application.name` or UNKNOWN, if the former is not defined.  

When a client registers with Eureka, it provides meta-data about itself — such as host, port, health indicator URL, home page, and other details. It does that by making a POST REST API call to the Eureka Server. The instance information is represented by `InstanceInfo` object. The `InstanceInfo` is sent to the Eureka server at regular intervals, starting at 40s after startup (configurable by `eureka.client.initialInstanceInfoReplicationIntervalSeconds`), and then onwards every 30s (configurable by `eureka.client.instanceInfoReplicationIntervalSeconds`). Note that this is different from sending heartbeats.

### Instance Heartbeat
After registering itself with Eureka Server, an instance sends heartbeats every 30s (configurable by `eureka.instance.leaseRenewalIntervalInSeconds`). If the server doesn’t receive a heartbeat, it waits 90s (configurable by `eureka.instance.leaseExpirationDurationInSeconds`) before removing the instance from registry and there by disallowing traffic to that instance. Sending heartbeat is an asynchronous task; if the operation fails, it backs off exponentially by a factor of 2 until a maximum delay of `eureka.instance.leaseRenewalIntervalInSeconds` * `eureka.client.heartbeatExecutorExponentialBackOffBound` is reached. There is no limit to the number of retries for registering with Eureka.  

### Health Check
By default, Eureka uses the client heartbeat to determine if a client is up. Unless specified otherwise, the client does not propagate the current health check status of the application, per the Spring Boot Actuator. Consequently, after successful registration, Eureka always announces that the application is in 'UP' state.  

By enabling Eureka health checks, the client application will send health status (the same health status reported by actuator/health endpoint of Actuator framework) to the Eureka server, and server will use this status to determine if server should advertise hostname/IP of this particular application to other applications.

### Fetch Registry
If `eureka.client.fetchRegistry` is true, the client fetches the Eureka server registry at startup and caches it locally. From then on, it only fetches the delta (this can be turned off by setting `eureka.client.shouldDisableDelta` to false, although that’d be a waste of bandwidth). The registry fetch is an asynchronous task scheduled every 30s (configurable by `eureka.client.registryFetchIntervalSeconds`). If the operation fails, it backs off exponentially by a factor of 2 until a maximum delay of `eureka.client.registryFetchIntervalSeconds` * `eureka.client.cacheRefreshExecutorExponentialBackOffBound` is reached. There is no limit to the number of retries for fetching the registry information.

### Client Cache
To reduce number of registry fetch requests made to the server, the client caches registry information on its end. This helps with availability in-case Eureka Server goes down. However it has the drawback that it can report services being available (for some time - till cache is refreshed) even though the service was down.

## Eureka Server
To run a server, use the `@EnableEurekaServer` annotation

In [ ]:
@SpringBootApplication
@EnableEurekaServer
public class Application {

    public static void main(String[] args) {
        new SpringApplicationBuilder(Application.class).web(true).run(args);
    }

}

### Peer Awareness
By default, every Eureka server is also a Eureka client and requires (at least one) service URL to locate a peer. In peer aware mode it replicates the service registry across other Eureka servers, to provide load balancing and resiliency. You can disable peer awareness by setting `eureka.client.register-with-eureka` to false.

When the eureka server starts up it tries to fetch all the registry information from the peer eureka nodes. This operation is retried 5 times for each peer (configurable by `eureka.server.numberRegistrySyncRetries`). If for some reason this operation fails, the server does not allow clients to get the registry information for 5min (configurable by `eureka.server.getWaitTimeInMsWhenSyncEmpty`).

### Self Preservation Mode
Self-preservation is a feature where Eureka servers stop expiring the client instances from the registry when they do not receive heartbeats (from peers and client microservices) beyond a certain threshold.  

Explained in detail [here](https://blog.thesoftwarefeed.com/p/the-mystery-of-eureka-self-preservation)

## Securing Eureka
We can secure client server communication using mTLS. In order to do so, we need to make sure that both client and server communicate over HTTPS. We make the following configuration:
```yml
# Eureka Server
server:
  port: ${PORT:8761}
  ssl:
    enabled: true
    key-store: "classpath:discovery_keystore.jks"
    key-store-password: pa$$w0rd
    key-store-type: PKCS12
    key-password: pa$$w0rd
    trust-store: "classpath:discovery_truststore.jks"
    trust-store-type: PKCS12
    trust-store-password: pa$$w0rd
    client-auth: need
```
The `client-auth` set to need means any client that wants to communicate to the server must pass in a certificate that is trusted by the server. The `discovery_keystore` and `discovery_truststore` files are generated in the following manner:
1. Generate the `discovery_keystore` store by running the following `keytool` command
```
keytool -genkey -alias discovery -keystore discovery_keystore.jks -keyalg RSA -keysize 2048
Enter keystore password:
What is your first and last name?
  [Unknown]:  localhost
What is the name of your organizational unit?
  [Unknown]:  Dev
What is the name of your organization?
  [Unknown]:  Stellar
What is the name of your City or Locality?
  [Unknown]:  Patna
What is the name of your State or Province?
  [Unknown]:  Bihar
What is the two-letter country code for this unit?
  [Unknown]:  IN
Is CN=localhost, OU=Dev, O=Stellar, L=Patna, ST=Bihar, C=IN correct?
  [no]:  y

Generating 2,048 bit RSA key pair and self-signed certificate (SHA256withRSA) with a validity of 90 days
        for: CN=localhost, OU=Dev, O=Stellar, L=Patna, ST=Bihar, C=IN
```
Then export certificate from the above generated keystore:
```
keytool.exe -export -alias discovery -keystore discovery_keystore.jks -file discovery.crt
```
2. Repeat the same for the client application (in this case gateway application)
```
keytool -genkey -alias gateway -keystore gateway_keystore.jks -keyalg RSA -keysize 2048
Enter keystore password:
What is your first and last name?
  [Unknown]:  localhost
What is the name of your organizational unit?
  [Unknown]:  Dev
What is the name of your organization?
  [Unknown]:  Stellar
What is the name of your City or Locality?
  [Unknown]:  Patna
What is the name of your State or Province?
  [Unknown]:  Bihar
What is the two-letter country code for this unit?
  [Unknown]:  IN
Is CN=localhost, OU=Dev, O=Stellar, L=Patna, ST=Bihar, C=IN correct?
  [no]:  y

Generating 2,048 bit RSA key pair and self-signed certificate (SHA256withRSA) with a validity of 90 days
        for: CN=localhost, OU=Dev, O=Stellar, L=Patna, ST=Bihar, C=IN
```
3. Similarly, generate `discovery_truststore` and `gateway_trustore`. These need not have any keypair or certificate in them right now.
4. Import `discovery.crt` into `gateway_truststore` and `gateway.crt` into `discovery_truststore`. This means that both client and server trust each others certificates.

The client application now needs to communicate over HTTPS and also present its certificate in order to communicate with the server:
```yml
# Eureka Application
server:
    port: ${PORT:9998}
    ssl:
        enabled: true
        key-store: "classpath:gateway_keystore.jks"
        key-store-password: pa$$w0rd
        key-store-type: PKCS12
        key-password: pa$$w0rd
        trust-store: "classpath:gateway_truststore.jks"
        trust-store-type: PKCS12
        trust-store-password: pa$$w0rd
```
During registration we need to inform Eureka server that our application’s endpoints are secured. To achieve it we should set property `eureka.instance.securePortEnabled` to true, and also disable non secure port, which is enabled by default with `eureka.instance.nonSecurePortEnabled` property. Doing so makes Eureka publish instance information that shows an explicit preference for secure communication. The Spring Cloud DiscoveryClient always returns a URI starting with https for a service configured this way.

## REST Service

## Eureka Client

To interact with Eureka server, Spring provides `EurekaClient`. By default, `EurekaClient` uses Spring’s `RestTemplate` for HTTP communication. Netflix provides an implementation of `EurekaClient` named `DiscoveryClient`.

Spring also provides a service discovery client named `DiscoveryClient` (not specific to Eureka). It provides a much simpler interface:

In [ ]:
public interface DiscoveryClient extends Ordered {
    // Gets all ServiceInstances associated with a particular serviceId
    List<ServiceInstance> getInstances(String serviceId);

    // returns all known service IDs.
    List<String> getServices();

    // other methods
}